In [3]:
from pydantic.v1 import BaseModel, Field
from langchain.tools import StructuredTool
from typing import List, Optional

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz
# import json
from prometheus_api_client import PrometheusConnect, MetricRangeDataFrame

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import io
import base64

In [2]:
class MetricValue(BaseModel):
    """Prometheus metric detail information object"""
    metric_value: float = Field(description="The value of the metric")
    timestamp: float = Field(description="The timestamp information associated with the metric value")


class MetricsResponse(BaseModel):
    """Prometheus metric response information object"""
    metric_name: str = Field(description="The name of the metric")
    metric_service_name: Optional[str] = Field(default=None, description="The service name associated with the metric")
    metric_namespace: Optional[str] = Field(default=None, description="The namespace name associated with the metric")
    metric_instance: Optional[str] = Field(default=None, description="The service name associated with the metric")
    metric_job_name: Optional[str] = Field(default=None, description="The job name associated with the metric")
    metric_pod_name: Optional[str] = Field(default=None, description="The pod name associated with the metric")
    metric_values: Optional[List[MetricValue]] = Field(default=None, description="The list of the metric value objects associated with the metric response")


class Metrics(BaseModel):
    """Filtered metrics information"""
    filter_name: str = Field(description="The name of the filter used to obtain the list of metric responses")
    filter_value: str = Field(description="The value of the filter used to obtain the list of metric responses")
    metrics: Optional[List[MetricsResponse]] = Field(default=[], description="The list of metric response objects")

In [3]:
class DateTimeValue(BaseModel):
    """Date Time Information Object"""
    timestamp: float = Field(description="The timestamp information associate with a date time")
    date_time_iso_format_string: str = Field(description="The ISO formatted string of the date time")
    timezone: str = Field(description="The timezone string information associated with the date time")

In [124]:
class MetricPlotFile(BaseModel):
    file_name: str = Field(description="The image plot file name")

In [ ]:
class InputSchemaGetTimeStamp(BaseModel):
    time_value: str = Field(description="It is either the string literal 'now' if the current time is needed, or the integer value of the desired time relative to the current time.")
    time_metric: str = Field(description="The string literal of the time metric. It is one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years'.")
    ago_flag: int = Field(description="The value of the ago flag. Can be either 0 or 1.")

In [ ]:
class InputSchemaQueryPrometheusMetrics(BaseModel):
    prom_service: str = Field(description="Then name of the Prometheus service.")
    prom_namespace: str = Field(description="The name of the namespace where the Prometheus service exists.")
    prom_port: str = Field(description="The value of the port where the Prometheus service is listening.")
    query_target_name: str = Field(description="The target name of the query.")
    query_target_value: str = Field(description="The value of the target query name.")

In [ ]:
class InputSchemaGetPrometheusMetricDataRange(BaseModel):
    prom_service: str = Field(description="Then name of the Prometheus service.")
    prom_namespace: str = Field(description="The name of the namespace where the Prometheus service exists.")
    prom_port: str = Field(description="The value of the port where the Prometheus service is listening.")
    metric_name: str = Field(description="The name of the metric whose values we want to retrieve")
    metric_range_start: str = Field(description="The start range timestamp of the time interval for the metric values retrieval.")
    metric_range_end: str = Field(description="The end range timestamp of the time interval for the metric values retrieval.")

In [ ]:
class InputSchemaPlotPrometheusMetricDataRangeAsFile(BaseModel):
    prom_service: str = Field(description="Then name of the Prometheus service.")
    prom_namespace: str = Field(description="The name of the namespace where the Prometheus service exists.")
    prom_port: str = Field(description="The value of the port where the Prometheus service is listening.")
    metric_name: str = Field(description="The name of the metric whose values we want to retrieve")
    metric_range_start: str = Field(description="The start range timestamp of the time interval for the metric values retrieval.")
    metric_range_end: str = Field(description="The end range timestamp of the time interval for the metric values retrieval.")

In [6]:
def get_timestamp(time_value: str, time_metric: str, ago_flag: str) -> DateTimeValue:
    tz = pytz.timezone('America/New_York')
    now = datetime.now(tz)

    if isinstance(time_value, str) and time_value == "now":
        dt = now
    else:
        unit = time_metric
        ago = int(ago_flag)
        # Ensure we have an integer
        value = int(time_value)
        dt = now
        if ago == 1:
            value = value*(-1)

        if unit == "seconds":
            dt += timedelta(seconds=value)
        elif unit == "minutes":
            dt += timedelta(minutes=value)
        elif unit == "hours":
            dt += timedelta(hours=value)
        elif unit == "days":
            dt += timedelta(days=value)
        elif unit == "weeks":
            dt += timedelta(weeks=value)
        elif unit == "months":
            dt += relativedelta(months=value)
        elif unit == "years":
            dt += relativedelta(years=value)
        else:
            raise ValueError(f"Unknown time unit: {unit}")

    timestamp = dt.timestamp()
    datetime_str = dt.isoformat()

    date_time = DateTimeValue(timestamp=timestamp,
                              date_time_iso_format_string=datetime_str,
                              timezone=str(tz))

    return date_time

In [7]:
def query_prometheus_metrics(prom_service: str, prom_namespace: str,
                             prom_port: str, query_target_name: str,
                             query_target_value: str) -> Metrics:
    prom_service_name = prom_service
    try:
        prometheus_url = f"http://{prom_service_name}.{prom_namespace}.svc.cluster.local:{prom_port}"

        # Connect to Prometheus
        prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)
        # print("Connected to Prometheus")

        metrics: List[MetricsResponse] = []

        # Define the query to get all metrics for the target namespace
        query = f'{{{query_target_name}="{query_target_value}"}}'
        all_metrics = prom.all_metrics(query)
        # print(f"All metrics:{all_metrics}")
        for item in all_metrics:
            metric_response = MetricsResponse(metric_name=item)
            metrics.append(metric_response)

        return Metrics(filter_name=query_target_name, 
                       filter_value=query_target_value, metrics=metrics)

    except Exception as e:
        print(f"Exception when fetching metrics: {e}")
        return Metrics(filter_name=query_target_name,
                       filter_value=query_target_value, metrics=[])

In [8]:
def get_prometheus_metric_data_range(prom_service: str, prom_namespace: str,
                                     prom_port: str, metric_name: str,
                                     metric_range_start: str,
                                     metric_range_end: str) -> Metrics:
    metric_response_list: List[MetricsResponse] = []
    prom_service_name = prom_service
    metric_range_start = float(metric_range_start)
    metric_range_end = float(metric_range_end)
    try:
        prometheus_url = f"http://{prom_service_name}.{prom_namespace}.svc.cluster.local:{prom_port}"

        # Connect to Prometheus
        prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)

        range_data = prom.get_metric_range_data(metric_name,
                                                start_time=datetime.fromtimestamp(metric_range_start),
                                                end_time=datetime.fromtimestamp(metric_range_end),
                                                )
        for item in range_data:
            metric_values: List[MetricValue] = []
            for item_value in item['values']:
                metric_value = MetricValue(timestamp=item_value[0],
                                           metric_value=item_value[1])
                metric_values.append(metric_value)
            metrics_response = MetricsResponse(metric_name=item['metric']['__name__'],
                                               metric_service=item['metric']['service'],
                                               metric_namespace=item['metric']['namespace'],
                                               metric_instance=item['metric']['instance'],
                                               metric_job_name=item['metric']['job'],
                                               metric_pod_name=item['metric']['pod'],
                                               metric_values=metric_values)
            metric_response_list.append(metrics_response)

        return Metrics(filter_name="namespace", filter_value=str(prom_namespace), metrics=metric_response_list)
    except Exception as e:
        print(f"Exception when fetching metric range data: {e}")
        return Metrics(filter_name="namespace", filter_value=str(prom_namespace), metrics=metric_response_list)

In [103]:
def plot_prometheus_metric_data_range_as_file(prom_service: str,
                                              prom_namespace: str,
                                              prom_port: str,
                                              metric_name: str,
                                              metric_range_start: str,
                                              metric_range_end: str) -> MetricPlotFile:

    prom_service_name = prom_service
    metric_range_start = float(metric_range_start)
    metric_range_end = float(metric_range_end)
    # resample_rate = input_params['resample_rate']
    try:
        prometheus_url = f"http://{prom_service_name}.{prom_namespace}.svc.cluster.local:{prom_port}"
        # Connect to Prometheus
        prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)
        # print("Connected to Prometheus")

        range_data = prom.get_metric_range_data(metric_name,
                                                start_time=datetime.fromtimestamp(metric_range_start),
                                                end_time=datetime.fromtimestamp(metric_range_end),
                                                )
        # print("Got ranged data")
        file_name = ""
        if (len(range_data) > 0):
            # Transform the data
            metric_df = MetricRangeDataFrame(range_data)
            metric_df = metric_df.resample('5s').ffill()
            metric_df["instantaneous_rate"] = metric_df["value"].diff() / metric_df.index.to_series().diff().dt.total_seconds()
            metric_df.fillna(0, inplace=True)
            metric_df['instantaneous_rate'] = metric_df['instantaneous_rate'].apply(lambda x : x if x > 0 else 0)

            # print("Got metric_df calculations")
            # Plot the irate values
            plt.figure(figsize=(10, 6))
            plt.plot(metric_df['instantaneous_rate'], label='irate')
            plt.xlabel('Time')
            plt.ylabel('Rate')
            plt.title(f'IRate of {metric_name}')
            plt.legend()
            plt.grid(True)
            # print("Got plot")

            file_name = f"FILE-plot-{metric_name}-{int(metric_range_start)}-{int(metric_range_end)}.png"
            plt.savefig(file_name, format='png')
            plt.close()
            # print("Saved image")

        else:
            # print("No data found!")
            plt.figure(figsize=(10, 6))
            plt.xlabel('Time')
            plt.ylabel('Rate')
            plt.title(f'IRate of {metric_name}')
            plt.grid(True)
            # print("Got empty plot")

            file_name = f"FILE-plot-{metric_name}-{int(metric_range_start)}-{int(metric_range_end)}.png"
            plt.savefig(file_name, format='png')
            plt.close()
            # print("Saved image")

        return MetricPlotFile(file_name=file_name)
    except Exception as e:
        print(f"Exception when plotting metric range data: {e}")
        return MetricPlotFile(file_name="")

In [104]:
#prom = PrometheusConnect(url="http://prometheus-operated.demo.svc.cluster.local:9090", disable_ssl=True)

In [119]:
#range_data = prom.get_metric_range_data("load_generator_total_msg",
#                                        start_time=datetime.fromtimestamp(1720122358.937599),
#                                        end_time=datetime.fromtimestamp(1720295156.060042),
#                                        )


## For long text or files, create the file locally and return an URL or filename, depending from where you will access it

### Also create two versions, one for ReAct and one for FuncCalling

### Note that when returning an answer use in the output a convention/template that you can then parse in the app before showing the answer in the UI
### For example you can have whenever returning a file the pattern: file_<img/csv/...>:<location/name>

In [9]:
prometheus_all_metrics_description = """
List available metric names in a Prometheus instance using an input filter.

:param prom_service: the name of the Prometheus service
:param prom_namespace: the name of the namespace where the Prometheus service resides.
:param prom_port: the port value number of the Prometheus service.
:param query_target_name: the name of the filter to use.
:param query_target_value: the value for the filter

:return: An object containing the available metric names.
"""

In [ ]:
# ALWAYS pass on values for all parameters as instructed.

In [10]:
# Create a tool for the agent
tool_prometheus_all_metrics = StructuredTool(
    name="List_Prometheus_Metrics_Names_Using_A_Filter",
    func=query_prometheus_metrics,
    args_schema=InputSchemaQueryPrometheusMetrics,
    description=prometheus_all_metrics_description,
    handle_tool_error=True,
    handle_validation_error=True,
)

In [11]:
prometheus_metric_data_range_description = """
List the application metric values and associated timestamps between a start and an end timestamp interval for a given metric name stored within a Prometheus instance.

:param prom_service: the Prometheus service name where the tool connects.
:param prom_namespace: the name of the namespace where the Prometheus service resides. 
:param prom_port: the port value number of the Prometheus service.
:param metric_name: the name of the application metric for which we want its values.
:param metric_range_start: the start value timestamp, which is a float number.
:param metric_range_end: the end value timestamp, which is a float number.

:return: An object containing the list of the desired application metric values and associated timestamp information.
"""

In [12]:
# Create a tool for the agent
tool_prometheus_metric_range = StructuredTool(
    name="List_metric_values_between_a_timestamp_range",
    func=get_prometheus_metric_data_range,
    args_schema=InputSchemaGetPrometheusMetricDataRange,
    description=prometheus_metric_data_range_description,
    handle_tool_error=True,
    handle_validation_error=True,
)

In [1]:
time_value_description = """
Calculate the timestamp, the iso formatted string and the timezone string of the requested time information. 

:param time_value: It is either the string literal 'now' if the current time is needed, or the integer value of the desired time relative to the current time
:param time_metric: It is extracted from the request is the string literal of the time metric: 'seconds', or 'minutes', or 'hours', or 'days' ,or 'weeks', or 'months', or 'years'. If undetermined, the time_metric value is 'seconds'.
:param ago_flag: It is 0 if the word ago was NOT present in the request, and it is 1 if the word ago was present in the request.

:return: The timestamp value, the ISO formatted string of the date time value, the timezone string.

Example: Input has '4 hours from now', then time_value=4, time_metric='hours' and ago_flag=0.
Example: Input has '3 hours ago', then time_value=3, time_metric='hours' and ago_flag=1.
Example: Input has 'current', then time_value='now', time_metric='seconds' and ago_flag=0.
"""

In [2]:
# Example 2: Input string contains '3 hours from now', then time_value=3, time_metric='hours' and ago_flag=0.
# len(time_value_description) # needs to be <1024 chars

1009

In [ ]:
"""
The time calculation requires three parameters. 
The first parameter is called time_value and its value is either the string literal 'now' if the current time is needed, or the integer value of the desired time relative to the current time.
The second parameter is called time_metric and its value is extracted from the request is the string literal of the time metric: 'seconds', or 'minutes', or 'hours', or 'days' ,or 'weeks', or 'months', or 'years'.
If the second parameter is not available in the request, then its value is 'seconds'.
The third parameter is called ago_flag and its value is 0 if the word ago was not present int the request, and its value is 1 if the word ago was present in the request.
Pass on the parameters as a dictionary that uses double quotes for the parameter names. ALWAYS pass on values for all parameters as instructed.

"""

In [14]:
# Create a tool for the agent
tool_time_value = StructuredTool(
    name="Get_timestamp_and_time_ISO",
    func=get_timestamp,
    args_schema=InputSchemaGetTimeStamp,
    description=time_value_description,
    handle_tool_error=True,
    handle_validation_error=True,
)

In [ ]:
plot_prometheus_metric_data_range_description_as_file = """
Creates a file with the plot of the instantaneous rate (irate) of an application metric values and associated timestamps between a start and an end timestamp interval for a given metric name stored within a Prometheus instance

:param prom_service: the name of the Prometheus service
:param prom_namespace: the name of the namespace where the Prometheus service resides.
:param prom_port: the port value number of the Prometheus service.
:param metric_name: the name of the application metric for which we want its instantaneous rate values plotted.
:param metric_range_start: the start value timestamp, which is a float number.
:param metric_range_end: the end value timestamp, which is a float number.

:return: The name of the file containing the plot.
"""

In [ ]:
#Pass on the parameters as a dictionary that uses double quotes for the parameter names.

In [ ]:
# Create a tool for the agent
tool_plot_prometheus_metric_range_as_file = StructuredTool(
    name="File_create_plot_irate",
    func=plot_prometheus_metric_data_range_as_file,
    description=plot_prometheus_metric_data_range_description_as_file,
    args_schema=InputSchemaPlotPrometheusMetricDataRangeAsFile,
    handle_tool_error=True,
    handle_validation_error=True,
    # return_direct=True
)